**Table of contents**<a id='toc0_'></a>    
- [Descripción del proyecto](#toc1_)    
  - [Objetivo](#toc1_1_)    
- [Descripción de datos](#toc2_)    
  - [Conclusión](#toc2_1_)    
- [Tratamiento, limpieza y enriquecimiento de datos](#toc3_)    
  - [Eliminación de Datos duplicados](#toc3_1_)    
  - [Tratamiento de Valores nulos](#toc3_2_)    
  - [Creación de nuevas columnas](#toc3_3_)    
  - [Conclusión](#toc3_4_)    
- [Creación de Tabla RFM](#toc4_)    
  - [Conclusión](#toc4_1_)    
- [EDA](#toc5_)    
- [Creación del modelo](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Descripción del proyecto](#toc0_)

En un mundo donde las empresas dependen cada vez más de los datos para tomar decisiones estratégicas, la capacidad de segmentar clientes de manera efectiva se vuelve crucial. La segmentación de clientes permite a las empresas personalizar sus estrategias de marketing, mejorar la retención de clientes y maximizar el valor del cliente a lo largo del tiempo.

## <a id='toc1_1_'></a>[Objetivo](#toc0_)
Este proyecto tiene como objetivo desarrollar una aplicación web interactiva que permita a los científicos de datos realizar análisis de segmentación de clientes utilizando el modelo RFM (Recencia, Frecuencia, Valor Monetario). La aplicación proporcionará herramientas para cargar datos, realizar análisis exploratorio y visualizar resultados de segmentación mediante gráficos interactivos.

# <a id='toc2_'></a>[Descripción de datos](#toc0_)

In [49]:
# Librerias
import pandas as pd
import datetime as dt
import json

In [3]:
# Obteniendo dataset
df = pd.read_csv('https://raw.githubusercontent.com/MaElmoon39/RetailSaviors/main/notebooks/datasets/Online_Retail.csv', 
                 encoding='unicode_escape')

# Visualización de dataset
df.head()

,INVOICE_NO,STOCK_CODE,DESCRIPTION,QUANTITY,INVOICE_DATE,UNIT_PRICE,CUSTOMER_ID,REGION
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2019 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2019 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2019 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2019 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2019 08:26,3.39,17850.0,United Kingdom


In [4]:
# Visualización de información
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   INVOICE_NO    541909 non-null  object 
 1   STOCK_CODE    541909 non-null  object 
 2   DESCRIPTION   540455 non-null  object 
 3   QUANTITY      541909 non-null  int64  
 4   INVOICE_DATE  541909 non-null  object 
 5   UNIT_PRICE    541909 non-null  float64
 6   CUSTOMER_ID   406829 non-null  float64
 7   REGION        541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
# Verificación de valores duplicados
print('Valores duplicados:', df.duplicated().sum())

# Visualización de ejemplo de duplicados
df[(df['DESCRIPTION']== 'UNION JACK FLAG LUGGAGE TAG') & (df['CUSTOMER_ID']== 17908.0)]

Valores duplicados: 5268


,INVOICE_NO,STOCK_CODE,DESCRIPTION,QUANTITY,INVOICE_DATE,UNIT_PRICE,CUSTOMER_ID,REGION
494,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,01/12/2019 11:45,1.25,17908.0,United Kingdom
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,01/12/2019 11:45,1.25,17908.0,United Kingdom


In [6]:
# Verificación de valores nulos
df.isna().sum()

INVOICE_NO           0
STOCK_CODE           0
DESCRIPTION       1454
QUANTITY             0
INVOICE_DATE         0
UNIT_PRICE           0
CUSTOMER_ID     135080
REGION               0
dtype: int64

In [7]:
#Visualización de cantidad de valores únicos
df.nunique()

INVOICE_NO      25900
STOCK_CODE       4070
DESCRIPTION      4214
QUANTITY          722
INVOICE_DATE    23260
UNIT_PRICE       1630
CUSTOMER_ID      4372
REGION             38
dtype: int64

In [8]:
# Descripcion del dataset
df.describe()

,QUANTITY,UNIT_PRICE,CUSTOMER_ID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [9]:
# Observación de valores negativos en unit_price
df[(df['UNIT_PRICE']< 0)]

,INVOICE_NO,STOCK_CODE,DESCRIPTION,QUANTITY,INVOICE_DATE,UNIT_PRICE,CUSTOMER_ID,REGION
299983,A563186,B,Adjust bad debt,1,12/08/2020 14:51,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,12/08/2020 14:52,-11062.06,NaN,United Kingdom


In [10]:
# Observación de valores negativos en quantity
df[(df['QUANTITY']< 0)]

,INVOICE_NO,STOCK_CODE,DESCRIPTION,QUANTITY,INVOICE_DATE,UNIT_PRICE,CUSTOMER_ID,REGION
141,C536379,D,Discount,-1,01/12/2019 09:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,01/12/2019 09:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,01/12/2019 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,01/12/2019 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,01/12/2019 10:24,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,09/12/2020 09:57,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,09/12/2020 10:28,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,09/12/2020 11:57,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,09/12/2020 11:58,1.25,17315.0,United Kingdom


## <a id='toc2_1_'></a>[Conclusión](#toc0_)
Nuestro dataser está formado por un total de 541909 filas y 8 columnas que representan los siguiente:

* `INVOICE_NO`: Número de orden
* `STOCK_CODE`: Código del producto
* `DESCRIPTION`: Descripción del producto
* `QUANTITY`: Cantidad de producto seleccionado
* `INVOICE_DATE`: Fecha de compra
* `UNIT_PRICE`: Precio unitario
* `CUSTOMER ID`: Número de cliente
* `REGION`: País de compra

Se cuentan con un total de 5268 valores duplicados, estos pueden ser eliminados dado que tenemos un ID de identificación. También se observan dos columnas con valores nulos. Observamos la cantidad de valores únicos de cada columna.Por último en la descripción de la tabla se aprecian valores negativos en las columnas `QUANTITY` con dos valores y `UNIT_PRICE` estos valores negativos pueden representar devoluciones o descuentos por lo que se dejaran tal cual están.

Las siguientes acciones a tomar son:

* Cambio por minusculas los nombres de las columnas.
* Eliminación de valores duplicados.
* Tratamiento de valores nulos.
* Creación de columna total y correción de tipo de datos.

Para ello vamos a realizar una copia de nuestra tabla original

# <a id='toc3_'></a>[Tratamiento, limpieza y enriquecimiento de datos](#toc0_)

## <a id='toc3_1_'></a>[Eliminación de Datos duplicados](#toc0_)

In [11]:
# copia del dataset
df_new = df.copy()

# camio a minúsculas de los nombres
df_new.columns = df_new.columns.str.lower()

# verificación de cambio
df_new.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'region'],
      dtype='object')

In [12]:
# Eliminación de valroes duplicados
df_new = df_new.drop_duplicates().reset_index(drop = True)

# Comprobando que no queden valores duplicados
df_new.duplicated().sum()

0

## <a id='toc3_2_'></a>[Tratamiento de Valores nulos](#toc0_)

Primero observaremos si podemos imputar la columna de **customer_id** para ello vamos a ver si podemos entcontrar sus valores usando la columna **invoice_no**, para ver si tenemos algún pedido relacionado con algún cliente.

In [14]:
# Adquisición de invoice donde se encuentra los customer_id con valores nulos
invoice_customer=df_new['invoice_no'][df_new['customer_id'].isna()]

# Tabla que contiene customer_id de acuerdo al invoice obtenido anteriormente
df_new['customer_id'][df_new['invoice_no'].isin(invoice_customer)].dropna().reset_index()

,index,customer_id


Podemos observar que no se tiene algún valor para imputar, al querer realizar una segmentación y no tener un ID conocido no podemos imputar con algún valor fijo ya que esto puede causar tener más peso a este tipo de clientes provocando segmentaciones erroneas, tampoco se le puede asignar algún valor aleatorio por que no se sabe si un cliente pudo haber realizado más de una compra.

Posiblemente esos valores ausentes pueden referirse a alguien que compró como una sesión de invitado por lo que no es parte de la segmentación que se quiere realizar, por lo que se decide eliminar estos valores a pesar de que represente una gran reducción en nuestros datos.

In [22]:
# Eliminación de valores nulos en la columna de customer ID
df_new = df_new[df_new['customer_id'].notna()]

# reducción de dataset
print(f'porcentaje reucido: {1-len(df_new)/len(df):.2%}')

# Verificación de eliminación de valores nulos
df_new.isna().sum()

porcentaje reucido: 25.89%


invoice_no      0
stock_code      0
description     0
quantity        0
invoice_date    0
unit_price      0
customer_id     0
region          0
dtype: int64

Al momento nuestro dataset se ha reducido en un 25%, aunque también hemos eliminado los problemas de valor nulo que teníamos con la columna **description**

## <a id='toc3_3_'></a>[Creación de nuevas columnas](#toc0_)

In [39]:
# Correción de tipo de datos
df_new['invoice_date'] = df_new['invoice_date'].astype('datetime64[ns]')
df_new['customer_id'] = df_new['customer_id'].astype('int')

# Creación de columna total
df_new['total'] = df_new['quantity'] * df_new['unit_price']

# Visualización de datos
df_new.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,region,total
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2019-01-12 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2019-01-12 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2019-01-12 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2019-01-12 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2019-01-12 08:26:00,3.39,17850,United Kingdom,20.34


In [40]:
# Visualizamos información del dataset.
df_new.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 401604 entries, 0 to 536640
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice_no    401604 non-null  object        
 1   stock_code    401604 non-null  object        
 2   description   401604 non-null  object        
 3   quantity      401604 non-null  int64         
 4   invoice_date  401604 non-null  datetime64[ns]
 5   unit_price    401604 non-null  float64       
 6   customer_id   401604 non-null  int32         
 7   region        401604 non-null  object        
 8   total         401604 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int32(1), int64(1), object(4)
memory usage: 29.1+ MB


In [41]:
df_new.describe()

,quantity,unit_price,customer_id,total
count,401604.000000,401604.000000,401604.000000,401604.000000
mean,12.183273,3.474064,15281.160818,20.613638
std,250.283037,69.764035,1714.006089,430.352218
min,-80995.000000,0.000000,12346.000000,-168469.600000
25%,2.000000,1.250000,13939.000000,4.250000
50%,5.000000,1.950000,15145.000000,11.700000
75%,12.000000,3.750000,16784.000000,19.800000
max,80995.000000,38970.000000,18287.000000,168469.600000


## <a id='toc3_4_'></a>[Conclusión](#toc0_)

Al hacer todo nuestro preprocesamiento de datos culminamos con un total de 401604 datos lo que represento una reducción de poco más de 25%, esto debido a que se eliminó los datos que contenían valores nulos en la columna de customer_id. Se decidieron eliminarlos por que no se podían imputar con la información que se tenía y no podía generar una segmentación erronea. 
Se creo una nueva columna llamada total para tener la cantidad comprada por producto, está columna nos servirá para la creación de nuestra tabla RFM. Con ello terminamos con un total de 9 columnas en lugar de 8

# <a id='toc4_'></a>[Creación de Tabla RFM](#toc0_)

vamos a crear una tabla con los valoresa RFM en donde cada sigla representa:

* Recency: Esta métrica nos dice la diferencia de días entre la diferencia entre la última fecha y la ultima fecha de compra*
 
* Frequency:Es la cantidad de compras que ha realizado el cliente calculada haciendo la suma de todas sus compras
 
* Monetary: Es la cantidad de dinero gastado sumando todas sus compras totales

*nota: para recency necesitamos aumentar en uno nuestra fecha máxima 

In [42]:
# Creación de nuestra última fecha
final_date = max(df_new['invoice_date']+ dt.timedelta(days=1))

In [44]:
# Creación de la tabla de rfm
rfm_df = df_new.groupby('customer_id').agg(
    recency = ('invoice_date', lambda x: (final_date- x.max()).days),
    frecuency = ('invoice_no', 'count'),
    monetary = ('total', 'sum')
)

# Visualización de tabla
rfm_df.reset_index()

,customer_id,recency,frecuency,monetary
0,12346,328,2,0.00
1,12347,41,182,4310.00
2,12348,77,31,1797.24
3,12349,20,73,1757.55
4,12350,313,17,334.40
...,...,...,...,...
4367,18280,161,10,180.60
4368,18281,5,7,80.82
4369,18282,94,13,176.60
4370,18283,11,721,2045.53


## <a id='toc4_1_'></a>[Conclusión](#toc0_)

Obtuvimos una nueva tabla en donde concentramos los valores RFM con un total de 4372 datos con 4 columnas

In [48]:
# Guardando tabla en archivo csv
df_sample = df_new.sample(20100)
df_sample.to_csv('datasets/sample_df', index= False)

In [50]:
# Generación del archivo en json
df_sample.to_json('datasets/sample_df.json', orient='records', lines=True)

# <a id='toc5_'></a>[EDA](#toc0_)

# <a id='toc6_'></a>[Creación del modelo](#toc0_)